In [1]:
# Creating the content of the data loading script
import os
import argparse
import pandas as pd
import pickle
def data_loading():
    '''
    Function for loading diabetes dataset
    '''
     #reading the data from its source
    data = pd.read_csv("https://raw.githubusercontent.com/Youngprof3/kubeflow_test/main/project_on_diabetes_kubeflow/data_loading/dataset.csv")
     #Save the data as a pickle file to be used by the preprocess component.
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f)



if __name__ == '__main__':
    data_loading()

In [2]:
# Creating the content of the data preprocessing script

import pandas as pd
import argparse
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def data_preprocessing():
    with open("data.pkl","rb") as f:
        data = pickle.load(f)
        #data features
        X = data.iloc[:,:-1]
        #target data
        y = data.iloc[:,-1:]
        #encoding the categorical columns
        le = LabelEncoder()

        X['gender'] = le.fit_transform(X['gender'])
        y['class'] = le.fit_transform(y['class'])

        #splitting the data
        X_train,X_test,y_train,y_test = train_test_split( X,y, test_size=0.3, random_state = 42)
        #feature scaling
        ms =MinMaxScaler(feature_range=(0,1))
        X_train = ms.fit_transform(X_train)
        X_test = ms.transform(X_test)

        #Save the train_data as a pickle file to be used by the train component.
        with open('train_data.pkl', 'wb') as f:
            pickle.dump((X_train,  y_train), f)

        #Save the test_data as a pickle file to be used by the predict component.
        with open('test_data.pkl', 'wb') as f:
            pickle.dump((X_test,  y_test), f)



if __name__ == '__main__':
    data_preprocessing()

C:\Users\user\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
# Creating the content of the model training script

import numpy as np
import pickle
import argparse
import os
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

def data_modelling():
    #loading the train data
    with open('train_data.pkl', 'rb') as f:
        train_data = pickle.load(f)
    #Separate the X_train from y_train.
        X_train, y_train = train_data

        log_regres = LogisticRegression(max_iter=10000)
        lr_model = log_regres.fit(X_train, y_train)

        svm = SVC(kernel ="linear", random_state=2)
        svm_model = svm.fit(X_train, y_train)

        knn = KNeighborsClassifier(n_neighbors=7)
        knn_model = knn.fit(X_train, y_train)


    with open('lr_model.pkl', 'wb') as f:
        pickle.dump(lr_model, f)

    with open('svm_model.pkl', 'wb') as f:
        pickle.dump(svm_model, f)

    with open('knn_model.pkl', 'wb') as f:
        pickle.dump(knn_model, f)


if __name__ == '__main__':
    data_modelling()

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-3-b11ae55df247>:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn_model = knn.fit(X_train, y_train)


In [4]:
# Creating the content of the model testing script

import numpy as np
import pickle
import argparse
import os


def model_testing():
    #loading the test data
    with open('test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    #Separate the X_test from y_test.
        X_test, y_test = test_data

    with open('lr_model.pkl', 'rb') as f:
        log_reg_model = pickle.load(f)

    with open('svm_model.pkl', 'rb') as f:
        svm_clas_model = pickle.load(f)

    with open('knn_model.pkl', 'rb') as f:
        knn_clas_model = pickle.load(f)

    y_pred_lr = log_reg_model.predict(X_test)
    y_pred_svm = svm_clas_model.predict(X_test)
    y_pred_knn = knn_clas_model.predict(X_test)

    with open('y_pred_lr.pkl', 'wb') as f:
        pickle.dump(y_pred_lr, f)

    with open('y_pred_svm.pkl', 'wb') as f:
        pickle.dump(y_pred_svm, f)

    with open('y_pred_knn.pkl', 'wb') as f:
        pickle.dump(y_pred_knn, f)



if __name__ == '__main__':
    model_testing()

In [10]:
# Creating the content of the model evaluation script

import numpy as np
import pickle
from sklearn.metrics import classification_report, confusion_matrix

def model_evaluation():
    #loading the test data
    with open('test_data.pkl', 'rb') as f:
        test_data = pickle.load(f)
    #Separate the X_test from y_test.
        X_test, y_test = test_data

    with open('y_pred_lr.pkl', 'rb') as f:
        y_pred_lr = pickle.load(f)
    with open('y_pred_svm.pkl', 'rb') as f:
        y_pred_svm = pickle.load(f)
    with open('y_pred_knn.pkl', 'rb') as f:
        y_pred_knn = pickle.load(f)


    class_report_lr = classification_report(y_test, y_pred_lr)
    class_report_svm = classification_report(y_test, y_pred_svm)
    class_report_knn = classification_report(y_test, y_pred_knn)


    print(f'Classification report for the logistic regression model: {class_report_lr}')

    print(f'Classification report for the support vector machine model: {class_report_svm}')

    print(f'Classification report for the k-nearest neighbor model: {class_report_knn}')


    confusion_matrix_lr = confusion_matrix(y_test, y_pred_lr)
    confusion_matrix_svm = confusion_matrix(y_test, y_pred_svm)
    confusion_matrix_knn = confusion_matrix(y_test, y_pred_knn)

    print(f'Confusion matrix for the logistic regression model: {confusion_matrix_lr}')

    print(f'Confusion matrix for the support vector machine model: {confusion_matrix_svm}')

    print(f'Confusion matrix for the k-nearest neighbor model: {confusion_matrix_knn}')
    
if __name__ == '__main__':
    model_evaluation()

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report for the logistic regression model:               precision    recall  f1-score   support

           0       0.92      0.97      0.95        36
           2       1.00      0.10      0.18        10
           3       0.97      1.00      0.98       253
           4       0.00      0.00      0.00         1

    accuracy                           0.96       300
   macro avg       0.72      0.52      0.53       300
weighted avg       0.96      0.96      0.95       300

Classification report for the support vector machine model:               precision    recall  f1-score   support

           0       0.88      1.00      0.94        36
           2       0.67      0.20      0.31        10
           3       0.98      1.00      0.99       253
           4       0.00      0.00      0.00         1

    accuracy                           0.97       300
   macro avg       0.63      0.55      0.56       300
weighted avg       0.96      0.97      0.96       300

Classificatio

In [6]:
# Install kfp.
%pip install kfp --quiet

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Turn functions to container operands.

import kfp
from kfp import dsl

def data_download_op():
    return dsl.ContainerOp(
        name='Ingest Data',
        image='arinze02/data_download:latest',
        command=["python", "data_loading.py"],
    )

def data_preprocessing_op():
    return dsl.ContainerOp(
        name='Preprocess Data',
        image='arinze02/data_preprocess:latest',
        command=["python", "data_preprocessing.py"],
    )

def model_train_op():
    return dsl.ContainerOp(
        name='Model Training',
        image='arinze02/model_train:latest',
        command=["python", "data_modelling.py"],
    )

def model_test_op():
    return dsl.ContainerOp(
        name='Model Testing',
        image='arinze02/model_test:latest',
        command=["python", "model_testing.py"],
    )

def model_evaluation_op():
    return dsl.ContainerOp(
        name='Model Evaluation',
        image='arinze02/model_evaluation:latest',
        command=["python", "model_evaluation.py"],
    )

In [12]:
@dsl.pipeline(
   name='Diabetes Classification Pipeline',
   description='Diabetes Classification Project.'
)
def Classification_Pipeline():
    pvc_op = dsl.VolumeOp(name='Persistent Volume Claim', 
            resource_name='data-volume',
            size='2Gi',
            modes=dsl.VOLUME_MODE_RWO)
            
    data_download_task = data_download_op().add_pvolumes({"/pvc_data": pvc_op.volume})

    data_preprocessing_task = data_preprocessing_op().add_pvolumes({"/pvc_data": pvc_op.volume}).after(data_download_task)

    model_training_task = model_train_op().add_pvolumes({"/pvc_data": pvc_op.volume}).after(data_preprocessing_task)

    model_testing_task = model_test_op().add_pvolumes({"/pvc_data": pvc_op.volume}).after(model_training_task)

    model_evaluation_task = model_evaluation_op().add_pvolumes({"/pvc_data": pvc_op.volume}).after(model_testing_task)

if __name__ == '__main__':
    kfp.compiler.Compiler().compile(Classification_Pipeline, 'Classification_Pipeline.yaml')